In [70]:
from PIL import Image
import numpy as np
import cv2
from scipy import signal as sig
from skimage.io import imread
from skimage.color import rgb2gray
from scipy import ndimage as ndi
from skimage.feature import corner_peaks
import matplotlib.pyplot as plt
from numpy import unravel_index
from HarrisDetection import harris_detection
import math

In [71]:
''' # 
    # @desc apply Gaussian Bluring on a given image
    # @params np.array img: Image to apply Guassian on it
    # @returns np.array I: np.array of blured image
    #          np.array I_norm: normalized array between (0,255) for blured Image 
'''
def gaussian_blur_scipy(img):
    kernel = np.array([[1, 2, 1],[2, 4, 2],[1, 2, 1]])
    I = sig.convolve2d(img, kernel, mode='same')/16.0
    I_norm = (I * 255 / I.max()).astype(np.uint8)
    return  I_norm

In [72]:
''' # 
    # @desc convert image to gray 
    # @params np.array img: Image to be converted to gray
    # @returns np.array gray image
'''
def imgtogray(img):
    img = np.array(img).astype('float64')
    img[:,:,0] *= 0.299
    img[:,:,1] *= 0.587
    img[:,:,2] *= 0.114
    return np.sum(img,axis=2).astype("uint8")

In [73]:
'''
    # @desc apply Sobel-Edge Detector on a given image
    # @params np.array img: Image to apply Guassian on it
    # @returns np.array Ix_norm: normalized array between (0,255) of edges in x direction of the image
    #          np.array Iy_norm: normalized array between (0,255) of edges in y direction of the image
    #          np.array Ixy_norm: normalized array between (0,255) of edges in x-y direction of the image
'''
def sobel_edge_detector_scipy(img):
    kernel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
    kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    Ix = sig.convolve2d(img, kernel_x, mode='same')
    Ix_norm = (Ix * 255 / Ix.max()).astype(np.uint8)
    Iy = sig.convolve2d(img, kernel_y, mode='same')
    Iy_norm = (Iy * 255 / Iy.max()).astype(np.uint8)
    grad = np.sqrt(Ix**2 + Iy**2)
    Ixy_norm = (grad * 255 / grad.max()).astype(np.uint8)
    return Ix_norm, Iy_norm, Ixy_norm

In [74]:
img = Image.open('HogTest.jpg')

In [75]:
img.size

(175, 300)

In [76]:
resized_img = img.resize((64,128))

In [77]:
np.array(resized_img).shape

(128, 64, 3)

In [78]:
gray = imgtogray(resized_img)

In [79]:
sobelx, sobely, magnitude = sobel_edge_detector_scipy(gray)

In [80]:
def get_angel(sobelx, sobely):
    angels = np.zeros(sobelx.shape)
    for x in range(sobelx.shape[0]):
        for y in range(sobelx.shape[1]):
            if sobelx[x,y] == 0:
                if sobely[x,y] != 0:
                    angels[x,y] = 90
            else: angels[x,y] = math.atan(sobely[x,y]/sobelx[x,y])
    return angels
    

In [81]:
angels = get_angel(sobelx, sobely)

In [82]:
angels.max()

90.0

In [83]:
pixels_per_cell = 8
bins_pre_cell = 9
histograms = []
for x in range (pixels_per_cell-1,edges.shape[0],pixels_per_cell):
    row = []
    
    for y in range (pixels_per_cell-1,edges.shape[1],pixels_per_cell):
        bins = [0]*9
        for pixelx in range(pixels_per_cell):
            for pixely in range(pixels_per_cell):
                bins[int(angels[x-pixelx,y-pixely]//(180//9))] += magnitude[x-pixelx,y-pixely]
        row.append(bins)
    histograms.append(row)
        
        

In [84]:
len(histograms[0])

8

In [97]:
def rss(a: np.ndarray) -> float:
    return np.linalg.norm(a)


In [100]:
normalized_histpograms = []
for y in range (1,len(histograms[0])):
    row = []
    for x in range (1,len(histograms)):
        cell = []
        cell.append(histograms[x-1][y-1])
        cell.append(histograms[x-1][y])
        cell.append(histograms[x][y-1])
        cell.append(histograms[x][y])
        cell = [item for sublist in cell for item in sublist]
        divider = rss(cell)
        cell /= divider
        row.append(cell)
    normalized_histpograms.append(row)
        
        
        
        

In [ ]:
# we can normalize image first to imrove the performance